# Generate Text with Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Anthropic's Claude model provided by Bedrock in this example. We will use the Bedrock version that is integrated with [LangChain](https://python.langchain.com/docs/get_started/introduction.html). LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example is called a zero-shot prompt because we are not providing any additional context other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment*.

#### Context
In the previous example `00-Bedrock_101.ipynb`, we explored how to use Boto3 client to communicate with Amazon Bedrock API. In this notebook, we will try to add a bit more complexity to leverage the LangChain framework for the similar use case. We will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate text with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![Architecture](./images/bedrock_langchain.jpg)

#### Use Case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email. We will use the Amazon Titan Text Large model using the Amazon Bedrock LangChain integration. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](./00-Bedrock_101.ipynb) notebook. ⚠️ ⚠️ ⚠️


In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Invoke the Bedrock client using LangChain Integration

Lets begin with creating an instance of Bedrock class from llms. This expects a `model_id` of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 `client` as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids under Amazon Bedrock.

Note that different models support different `model_kwargs`.

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

By passing the `client` in to LangChain, we should be able to ensure that the library uses the same boto3 client we checked the configuration of earlier:

In [ ]:
print(boto3_bedrock)
print(textgen_llm.client)

LangChain has abstracted away the Amazon Bedrock API and made it easy to build use cases. You can pass in your prompt and it is automatically routed to the appropriate API to generate the response. You simply get the text output as-is and don't have to extract the results out of the response body.

### Zero-Shot Text Generation
Let's prepare the prompt to generate an email for the Customer Service Manager to send to the customer.

In [ ]:
response = textgen_llm("""Human: Write an email from Bob, Customer Service Manager, 
to the customer "John Doe" that provided negative feedback on the service 
provided by our customer support engineer.

Assistant: """)

print_ww(response)

### Contextual Generation
By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

Previously we hardcoded the prompt, it might be the case that you have multiple customers sending similar negative feedback and you now want to use each of those customer's emails and respond to them with an apology but you also want to keep the response a bit personalized. In the following cell we are exploring how you can create a `PromptTemplate` to achieve this pattern.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant: """
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )

response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

### Text Summarization

Now, you can use any short text whose tokens are less than the maximum token of a foundation model. As an exmple of short text, let's take one paragraph of an [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) about announcement of Amazon Bedrock.

The prompt starts with an instruction `Please provide a summary of the following text.`. 

In [ ]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

In [ ]:
response = textgen_llm(prompt)
print_ww(response)

## Conclusion
You have now experimented with using `LangChain` framework which provides an abstraction layer on Amazon Bedrock API. Using this framework you have seen the usecase of generating an email responding to a customer due to their negative feedback and also summarizing a piece of AWS blog post.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the different parameters to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You